In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [2]:
dataset_file = '../datasets/Auto.csv'
data = pd.read_csv(dataset_file, index_col='name')
data = data.loc[data.horsepower != '?']
data.horsepower = data.horsepower.apply(int)
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
name,,,,,,,,
chevrolet chevelle malibu,18.0,8,307.0,130,3504,12.0,70,1
buick skylark 320,15.0,8,350.0,165,3693,11.5,70,1
plymouth satellite,18.0,8,318.0,150,3436,11.0,70,1
amc rebel sst,16.0,8,304.0,150,3433,12.0,70,1
ford torino,17.0,8,302.0,140,3449,10.5,70,1


In [28]:
train_data = data.sample(frac=.5, random_state=1)
test_data = data.drop(train_data.index)
train_X = train_data.horsepower.values.reshape(-1, 1)
train_X_2 = np.hstack([train_X, train_X ** 2])
train_X_3 = np.hstack([train_X, train_X ** 2, train_X ** 3])
test_X = test_data.horsepower.values.reshape(-1, 1)
test_X_2 = np.hstack([test_X, test_X ** 2])
test_X_3 = np.hstack([test_X, test_X ** 2, test_X ** 3])
train_y = train_data.mpg
test_y = test_data.mpg

In [29]:
model = LinearRegression().fit(train_X, train_y)
(test_y - model.predict(test_X)).pow(2).mean()

25.01423339941708

In [30]:
model2 = LinearRegression().fit(train_X_2, train_y)
(test_y - model2.predict(test_X_2)).pow(2).mean()

20.942287489569363

In [31]:
model3 = LinearRegression().fit(train_X_3, train_y)
(test_y - model3.predict(test_X_3)).pow(2).mean()

20.87712085084548

In [67]:
train_data = data.sample(frac=.5, random_state=5)
test_data = data.drop(train_data.index)
train_X = train_data.horsepower.values.reshape(-1, 1)
train_X_2 = np.hstack([train_X, train_X ** 2])
train_X_3 = np.hstack([train_X, train_X ** 2, train_X ** 3])
test_X = test_data.horsepower.values.reshape(-1, 1)
test_X_2 = np.hstack([test_X, test_X ** 2])
test_X_3 = np.hstack([test_X, test_X ** 2, test_X ** 3])
train_y = train_data.mpg
test_y = test_data.mpg

In [68]:
model = LinearRegression().fit(train_X, train_y)
print((test_y - model.predict(test_X)).pow(2).mean())
model2 = LinearRegression().fit(train_X_2, train_y)
print((test_y - model2.predict(test_X_2)).pow(2).mean())
model3 = LinearRegression().fit(train_X_3, train_y)
print((test_y - model3.predict(test_X_3)).pow(2).mean())

20.44845821786757
15.436639885482204
15.353884023392903


In [62]:
loocv_error = 0
for i in range(test_X.shape[0]):
    model = LinearRegression().fit(np.delete(train_X, i).reshape(-1, 1), np.delete(train_y.to_list(), i))
    loocv_error += ((np.delete(test_y.to_list(), i) - model.predict(np.delete(test_X, i).reshape(-1, 1))) ** 2).mean()
loocv_error / train_X.shape[0]

20.191528029182596

In [79]:
loocv_error = 0
for i in range(test_X_2.shape[0]):
    model = LinearRegression().fit(np.delete(train_X_2, i, axis=0), np.delete(train_y.to_list(), i))
    loocv_error += ((np.delete(test_y.to_list(), i) - model.predict(np.delete(test_X_2, i, axis=0))) ** 2).mean()
loocv_error / train_X_2.shape[0]

11.499112188721568

In [80]:
loocv_error = 0
for i in range(test_X_3.shape[0]):
    model = LinearRegression().fit(np.delete(train_X_3, i, axis=0), np.delete(train_y.to_list(), i))
    loocv_error += ((np.delete(test_y.to_list(), i) - model.predict(np.delete(test_X_3, i, axis=0))) ** 2).mean()
loocv_error / train_X_3.shape[0]

11.438497535998787

In [81]:
from sklearn.model_selection import KFold

In [97]:
cv = KFold(10, shuffle=True, random_state=1)

In [98]:
cv_error = 0
X = train_X
y = train_y
for train, test in cv.split(X):
    model = LinearRegression().fit(X[train], y[train])
    cv_error += ((y[test] - model.predict(X[test])) ** 2).mean()
cv_error / 10

28.58442114404799

In [99]:
cv_error = 0
for train, test in cv.split(X):
    model = LinearRegression().fit(train_X_2[train], y[train])
    cv_error += ((y[test] - model.predict(train_X_2[test])) ** 2).mean()
cv_error / 10

23.99543085741586

In [100]:
cv_error = 0
for train, test in cv.split(X):
    model = LinearRegression().fit(train_X_3[train], y[train])
    cv_error += ((y[test] - model.predict(train_X_3[test])) ** 2).mean()
cv_error / 10

24.531021279265218

In [ ]:
dataset_file = '../datasets/'